In [ ]:
# EficiencyExperiment = True
# SurrogateExperiment = True

# MAIN = "" # MAIN DIRECTORY #

### IMPORTS

In [ ]:
#Requirements made with pipreqs or pipreqsnb
import os
import math
import scipy
import time
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
import pandas as pd

In [ ]:
## Here the directories for the results are generated, some folders are made on the directory were the folder Notebooks is ##

def parent(path):
    return os.path.split(path)[0]
def sign(x):
    if x:
        return abs(x)/x
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

path = os.getcwd()
os.system("mkdir %s"%(os.path.join(parent(path),"Results"))) #Here results of the main experiments are stored
os.system("mkdir %s"%(os.path.join(parent(path),"Surrogate"))) #Here results of the surrogate experiment are stored

### Define error function, distance and label assignment

In [ ]:
## Some important functions and Distances are defined for experiments ##

def compute_labs(x,centers): #Given a set of points and a set of centroids returns a list of labels
    labs = np.zeros(shape=(x.shape[0]))
    for i in range(x.shape[0]):
        labs[i]=np.argmin(np.linalg.norm((x[i,:]-centers),axis=1))
    return (labs)

def distance(x,centers,compute_labs = False): #Computes a list of the distances from each point to each nearest centroid, if requested also returns a list of labels
    dis = np.zeros(shape=(x.shape[0]))
    if compute_labs:
        labs = np.zeros(shape=(x.shape[0]))
        for i in range(x.shape[0]):
            aux = np.linalg.norm((x[i,:]-centers),axis=1)
            dis[i] = aux.min()
            labs[i] = np.argmin(aux)
        return (dis,labs)
    else:
        for i in range(x.shape[0]):
            dis[i] = np.linalg.norm((x[i,:]-centers),axis=1).min()
        return (dis)

def distance_labs(x,centers,labs): #This functions does the same but the labels are allready given, which speeds up computations
    dis = np.zeros(shape=(x.shape[0]))
    for i in range(x.shape[0]):
            dis[i] = np.linalg.norm((x[i,:]-centers[labs[i],:]))
    return (dis)

def inertia(x,centers,weight = np.array([None]),labs = np.array([None])): #Returns the K-means error(weighted if weight is given) for a dataset and centroids
    if any(weight==None):
        weight=np.ones(shape=(x.shape[0]))
    if any(labs!=None):
        error = np.sum(weight*(distance_labs(x,centers,labs))**2)
    else:
        error = np.sum(weight*(distance(x,centers))**2)
    return (error/np.sum(weight))

def print_and_log(log_file,text): #If wanted to log the experiment steps, this function prints a text to the log file and to the screen
    print(text,file=log_file)
    os.system("echo "+str(text))
    
def max_order(a): #Helps debugging, returns the maximum order of magnitude(in base 10) of the values in an array
    return round(np.log(a.max()+1.0e-10)/np.log(10))

### Read datasets

In [ ]:
## Here an object is defined in order to read datasets ##

class Dataset:
    
    "Data reader"
    #File in .csv format, a single file can be given or a folder containing many datasets(with the same number of columns). Variables should be numerical, so in order to remove 
    #class variables or any non interesting variables insert a list of indexes corresponding to the columns to be removed. If the data was already stored as a Python object, it can be stated with the last input variable
    
    def __init__(self,file = None,folder = None,class_index = [],header = "infer",is_python_object=False): 
        
        if is_python_object:
            with open(file, 'rb') as input:
                self.data= pickle.load(input)
        else:
            if file:
                df = pd.read_csv(file,header = header) #Reads .csv file using panda's dataframe
                #df = df.fillna(axis = 1,method = "bfill") #If there are missing values uncomment this line, and use the desired filling method
                self.classes = df[df.columns[class_index]] #Creates the list of columns to be removed
                df.drop(labels = df.columns[class_index],axis = 1,inplace = True) #Remove the undesired columns
                self.names = list(df.columns) 
                self.data = df.to_numpy()                
                del df #Delete dataframe
                
            #In this case the same procedure is conducted, but a list of .csv files are concatenated instead
            elif folder:
                df = pd.DataFrame()
                for f in os.listdir(folder):
                    if ".csv" in f:
                        df = df.append(pd.read_csv(os.path.join(folder,f),header = header))
                #df = df.fillna(axis = 1,method = "bfill")
                self.classes = df[df.columns[class_index]]
                df.drop(labels = df.columns[class_index],axis = 1,inplace = True)
                self.names = list(df.columns)
                self.data = df.to_numpy()
                del df

### Create Concept Drift

In [ ]:
## Here the function that generated controlled (1+epsilon)-drifts is given ##

#The number of clusters must be given in order to compute a set of centroids. The number of concepts that must be generated can be given, if the e_list contains more than one value then the concepts are generated according 
#to the list,but if e_list has a single value then n_concepts are generated with the epsilon value given on the list. The file/folder of the original dataset is given, and where the output streaming data is stored must be given.
#The tolerance of the heuristic procedure is set default to 0.05, and the class_index must be given as well as when a dataset was read.

def Generate_concept(K = None, n_concepts = None, e_list= [None],file_in = None, file_out = None, tol = 0.05,class_index = []):
    
    if len(e_list)==1:
        aux = e_list
        e_list = [aux[0] for _ in range(n_concepts)]
        del aux
    else:
        n_concepts = len(e_list) #If e_list has more than one value then n_concept is the size of the list

    #The dataset is read
    if os.path.isfile(file_in):
        data = Dataset(file = file_in,class_index = class_index).data
    else:
        data = Dataset(folder = file_in,class_index = class_index).data
    
    
    data = data[np.random.shuffle(np.arange(data.shape[0])),:][0] #Shuffle the data
    if data.shape[0]>100000: #For massive data, select only 100.000 data points maximum
        data = data[0:100000,:]
    
    init_tol = tol #The tolerance will increase as more iterations are run, in order to generate data in reasonable time
    cd_size = data.shape[0] #The concept drifts will be generated with the whole original dataset, then the number of points on each concept is the number of points from the original dataset

    i = 0
    current_data = data[0:cd_size,:]
    out_data = current_data
    
    skm = KMeans(n_clusters = K,n_init = 1,init = "k-means++",max_iter = 100) #Define the K-means problem
    skm.fit(current_data) #Compute the centroids C_{1}
    
    for e in e_list:
        i+=1
        
        current_error = skm.inertia_/current_data.shape[0] #Compute E(X_{i-1},C_{i-1})
        current_centers = skm.cluster_centers_ #C_{i-1}
        labels = skm.labels_
        
        valid = False
        
        maximum_random = 50 #Maximum of random directions considered
        maximum_mag = 20 #Maximum number of magnitudes computed(alpha^*_i)
        j = 1
        tol = init_tol #Set tolerance to the initial value
        
        while j<maximum_random and not valid:
            
            random = np.random.random(size=(K,current_data.shape[1])) #Set a random direction for each cluster
            rand = np.divide(random,np.linalg.norm(random,axis = 1).reshape(random.shape[0],1)) #Normalize directions
            alpha = math.sqrt(e*current_error/(K*cd_size)) #Set initial value for alpha
            j+=1
            k = 0
            while k<maximum_mag and not valid:
                
                k+=1
                random = alpha*rand #Scale random directions
                traslated_data = current_data.copy()
                
                for lab in range(K):
                    traslated_data[np.where(labels==lab),:] = traslated_data[np.where(labels==lab),:] + random[lab,:] #Traslate each cluster
                
                new_error = inertia(traslated_data,current_centers) #Compute the new error
                print("Error ratio= %s"%(new_error/current_error)) #For log
                per = ((new_error/current_error-1)-e)/e #Compute ratio with respect to the desired error
                print(per*100)
                
                if (abs(per)>1 and k>2): #If the error differs more than a 100% with respect to the desired one, compute other random directions
                    print("Change direction")
                    break
                if abs(per)<tol: #If the difference is below the threshold, then it is a valid traslation and the loop is ended
                    valid = True                    
                else:
                    alpha-=(new_error/((1+e)*current_error)-1)*math.sqrt(e*current_error/(K*cd_size)) #If the difference is not below the threshold, update alpha 
            tol+=0.01
        if valid:
            print("Success")
            skm = KMeans(n_clusters = K,init = skm.cluster_centers_,max_iter = 100)
            skm.fit(traslated_data) #Compute new centroids C_{i}
            out_data = np.concatenate((out_data,traslated_data)) #Concatenate the generated data
            current_data = traslated_data.copy()
        else:
            print("Not succeded")
        print(e)
    
    with open(file_out, 'wb') as output:
        pickle.dump((cd_size,out_data), output, pickle.HIGHEST_PROTOCOL) #Save the generated Streaming data as a python object
    
    #Remove data 
    del data 
    del current_data
    del traslated_data
    del out_data

In [ ]:
## Here some dictionaries and lists are used to determine how to generate the streaming data for each dataset ##

#A dict containing the class_indexes for each dataset
index_dict = {"Urban":[],"Google":[0],"Postures":[0,1],"SUSY":[0],"Gas":[0],"Pulsar":[8],"MiceProtein":[0,-4,-3,-2,-1],"Frogs":[0,-4,-3,-2,-1],"Epilepsia":[0,-1],"Gesture":[-2,-1]} 
#A list of used number of clusters to generate different streaming data
k_list = [5,10,25,50]
#A dictionary with the empirical optimal values of K for each dataset
k_dict = {"SUSY":2,"Pulsar":2,"Epilepsia":5,"Urban":400,"Frogs":60, "Google": 10,"MiceProtein":10,"Gesture":10,"Gas":5}
#This boolean determines if the number of clusters is used from the list or using the dictionary
k_from_list = True
#The names of the dataset used to generate different streamming data
names = ["Urban","SUSY","Pulsar","Epilepsia","Google","Frogs","Gesture","Gas"]#"MiceProtein"
#Dictionary containing the input file/folder of each dataset
direc_dict = {"Urban":"urbanGB.txt","Google":"google_review_ratings.csv","Postures":"Postures.csv","Frogs":"Frogs_MFCCs.csv","SUSY":"SUSY.csv","Gas":"gas-sensor-array-temperature-modulation","Gesture":"gesture_phase_dataset","Pulsar":"HTRU_2.csv","MiceProtein":"MiceProtein.csv","Epilepsia":"Epilepsia.csv"}
#List of epsilon values used, for this experiment n concepts are generated with the same epsilon value, hence each value defined in this list determines different experiments
e_list = [0.5,1,2]
#Dictionary determining the string representation of the epsilon values used on the name when saving the output
e_dict = {0.5:"05",1:"1",2:"2"}

In [ ]:
generate = False #Whether to generate or not

if generate:
    for k in k_list:
        for e in e_list:
            for name in names:
                if not k_from_list:
                    k = k_dict[name]
                Generate_concept(K = k, n_concepts = 9, e_list= [e],file_in = os.path.join(parent(path),"Datasets",direc_dict[name]), 
                                 file_out = os.path.join(parent(path),"Python Objects",name+e_dict[e]+"k="+str(k)), tol = 0.05,class_index = index_dict[name])
                print("Done with %s for K=%s and e=%s\n"%(name,k,e))

#Thse lines below define isolated generations
#e_list = [0.5]#[0.1,0.2,0.5,1,2,10]
#Generate_concept(k = 13, n_concepts = 9, e_list= e_list,file_in = os.path.join(MAIN,"Datasets/gas-sensor-array-temperature-modulation"), file_out = os.path.join(MAIN,"Python Objects/SimulatedData05"), tol = 0.1)
#Generate_concept(k = 1, n_concepts = 1, e_list= e_list,file_in = os.path.join(MAIN,"Datasets/gas-sensor-array-temperature-modulation"), file_out = os.path.join(MAIN,"Python Objects/SimulatedData1"), tol = 0.06)
#Generate_concept(k = 13, n_concepts = 9, e_list= e_list,file_in = os.path.join(MAIN,"Datasets/SUSY.csv"), file_out = os.path.join(MAIN,"Python Objects/AritzCDSUSY"), tol = 0.05)

### Stream generator

In [ ]:
## This Pyhton object generates an iterator that returns batches of data with controlled concept drift ##

class StreamCD:

    """Iterator that returns batches of data"""
    
    #Input file with the streaming data(in our case a python object), determine the batch size and the desired period of each concept drift. 
    #First batches file contains the original dataset and is used to burn out the stored batches before making measurements, tmax determines 
    #how many batches untill the burn out. Class index is used again for the original dataset.
    
    def __init__(self,file = "",batch_size = None,cd_period = None,first_batches="",tmax = 0,class_index = []):
        
        with open(os.path.join(file), 'rb') as input:
            
            (concept_size,data)=pickle.load(input) #Load streaming data with the concept drift size(original data size)
        
        if first_batches: #If stated, read first batches
            if os.path.isfile(first_batches):
                self.initial_data = Dataset(file = first_batches,class_index = class_index).data
            else:
                self.initial_data = Dataset(folder = first_batches,class_index = class_index).data
                
        self.tmax = tmax
        self.cd_size = concept_size
        self.data = data
        self.batch_size = batch_size
        self.max = data.shape[0]
        self.num = 0
        self.concept = 0
        self.cd_period = cd_period
        self.i = 0
        self.initial = True
        self.swap = True
        
        if self.cd_period and not self.batch_size: #If the period is stated but not the batch size then compute it such that the maximum number of data is extracted
            self.batch_size = self.cd_size//self.cd_period
            
        if self.batch_size*self.cd_period>self.cd_size: #The data is extracted in order without replacement, so we can not extract more data than we have
            print("Imposible")
            raise StopIteration

    def __iter__(self):
        return self

    def __next__(self):
        
        cd = False
        if self.i>self.tmax and self.swap: #If tmax batches have already passed, swap to the streaming data with concept drifts
                self.initial = False
                self.num = 0
                self.i = 0
                self.swap = False
                
        if self.num+self.batch_size>self.initial_data.shape[0]: #If more batches are needed from the original data until the burn out, then restart from the beginning
            self.num = 0
                
        if self.initial: #Return batches from the original dataset
            batch = self.initial_data[self.num:self.num+self.batch_size,:]
            self.num+=self.batch_size
            self.i+=1
        
        if not self.initial: 
            
            if self.cd_period:#If cd_period is stated, then return batches with concept drift each period
                if self.i == self.cd_period:
                    self.concept+=1
                    self.num = self.concept*self.cd_size
                    self.i = 0
                    cd = True

                if self.num>=self.data.shape[0]: #If all batches have been returned then stop iteration
                    raise StopIteration

                self.i+=1
                aux = self.data[self.concept*self.cd_size:(self.concept+1)*self.cd_size,:]
                np.random.seed(int(10000*time.time())%(2**31))
                batch = aux[np.random.choice([i for i in range(aux.shape[0])],self.batch_size,False),:] #Choose a batch randomly
                self.num+=self.batch_size

            else:#If cd_period is not stated, then return as many batches as possible from each concept 

                if self.num//self.cd_size != self.concept:
                    self.concept = self.num//self.cd_size
                    cd = True
                last_index = (self.concept+1)*self.cd_size

                if self.num>=self.data.shape[0]:
                    raise StopIteration

                if self.num+2*self.batch_size>last_index-1:
                    batch = self.data[self.num:self.num+self.batch_size,:]
                    self.num+=last_index

                else:
                    batch = self.data[self.num:self.num+self.batch_size,:]
                    self.num+=self.batch_size
        
        return batch,cd #Return a batch and whether a concept drift has occurred or not
        

### Streaming K MEans

In [ ]:
## Define some useful functions for the experiments ##

def dis_init_kmplus(n,k): #Returns the number of computed distances when initializing with KM++
    return n*k*(k-1)/2-((k-1)**3)/3-((k-1)**2)/2-(k-1)/6

def Mean(x,ax): #Computes the mean value through an axis(ax)
    x = x
    if x.shape[ax]:
        return np.sum(x,axis=ax)/x.shape[ax]
    else:
        return np.zeros(x.shape[ax-1])
    
def kplus(X, K,w = [False]): #Computes the KM++ initialization, returns a set of initial Centroids
    np.random.seed(2020)            #Fixing seeds, so that every initialization(UPC,ICB,HI and WKI) computes the same C^0
    C = [X[np.random.choice(range(X.shape[0])),:]]
    for k in range(1, K):
        if np.array(w).any():
            D2 = w*np.array([min([np.inner(c-x,c-x) for c in C]) for x in X])
        else:
            D2 = np.array([min([np.inner(c-x,c-x) for c in C]) for x in X])
        probs = D2/D2.sum()
        cumprobs = probs.cumsum()
        r = np.random.random()
        for j,p in enumerate(cumprobs):
            if r < p:
                i = j
                break
        C.append(X[i,:])
    return np.array(C)

def count(labels,k): #Returns two lists, the first one with all the labels ordered, and the second one with how many of each label is in the input list labels(ordered)
    
    aux_u,aux_count = np.unique(labels,return_counts=True)
    if len(aux_u) == k:
        return aux_u,aux_count
    else: #If not every label is present, then 0s need to be added to the count list
        u = list(range(k))
        count = list(range(k))
        for lab in u:
            if lab in aux_u:
                count[lab] = aux_count[np.where(aux_u == lab)][0]
            else:
                count[lab] = 0
        return u,count

In [ ]:
## Object where the Streaming K-means is carried out, with different methods FSKM(with different initializations) and PSKM ##

class SkMeans:
    
    #Takes an initial batch a executes KM over it, with k clusters. Afterwards FSKM with forget parameter rho or PSKM is executed. With a threshold given, maximum number
    #of stored batches is determined with rho, or viceversa. The method determines which initialization technique to use when FSKM is used, or whether to use PSKM. The first 
    #initialization method is KM++ by default. If wanted to run in multiple threads the number of 
    #used cores is determined by jobs(default 1). n_init determines if the initialization shall be repeated. mini_batch initialization ban also be used. The variable plot accepts 
    #a list containing "pca" or "mds", which will plot a PCA or MDS 2D projection of the datapoints, cmap determines the colormap used for this plot.
    
    def __init__(self, initial_batch = None, k = 2, rho = None, thresh = None,maxBatch = None,method = None, 
                 first_init = "k-means++",chain_length = 200,afkmc2 = False,jobs = None,n_init = 1,mini_batch_size = None, plot = [""], cmap = "hsv"):
        
        ### Save important parameters
        self.k = k
        self.first_init = first_init
        self.jobs = jobs
        self.n_init = n_init
        self.mini_batch_size = mini_batch_size
        self.chain_length = chain_length
        self.afkmc2 = afkmc2
        self.method = method
        self.dis_init = [0]
        
        if maxBatch == None:
            self.maxBatch = math.ceil(math.log(thresh)/math.log(rho)) #If maxBatch was not given then calculate it
        elif rho == None:
            rho = thresh**(1./maxBatch) #If rho was not given then calculate it
            self.maxBatch = maxBatch
        self.rho = np.array([rho**t for t in range(self.maxBatch,-1,-1)])
        self.thresh = thresh
        self.batches = [initial_batch] #Initiate array of batches
        
        ### Choose the kmeans method
        if self.mini_batch_size:
            self.kmeans = MiniBatchKMeans(n_clusters = self.k,n_init = self.n_init,init = self.first_init,batch_size = self.mini_batch_size)
        elif self.first_init == "kmc2":
            self.kmeans = KMeans(n_clusters = self.k,n_init = self.n_init,init = kmc2(np.vstack(self.batches),
                            k = self.k, chain_length=self.chain_length, afkmc2=self.afkmc2), n_jobs = self.jobs)
            self.dis_init[-1]+= dis_init_kmc2(m = self.chain_length,k = self.k)
        else:
            init_centers = kplus(self.batches[-1],self.k)
            self.error_real_init = [inertia(x = self.batches[-1],centers = init_centers)]
            self.kmeans = KMeans(n_clusters = self.k,init = init_centers,n_jobs = self.jobs)
            self.dis_init[-1]+= dis_init_kmplus(n = self.batches[0].shape[0],k = self.k)
        
        ### Fit to first batch
        start = time.time()
        self.kmeans.fit(self.batches[0],
                        sample_weight = np.array([self.rho[i] for i in range(-len(self.batches),0,+1) for _ in range(self.batches[i].shape[0])]))
        self.time = [time.time()-start]
        
        ### Save results
        self.centers = [self.kmeans.cluster_centers_]
        self.errors = [self.kmeans.inertia_]
        self.error_ratio = [0]
        self.t = [0]
        self.lloyd_iter = [self.kmeans.n_iter_]
        self.dis_lloyd = [self.kmeans.n_iter_*self.k*sum([batch.shape[0] for batch in self.batches])]
        self.dis = [self.dis_lloyd[0]+self.dis_init[0]]
        self.drift = [False]
        self.cardinalities = np.zeros(shape = (self.k))
        self.batch_card = np.zeros(shape = (self.k))
        self.mean = np.zeros(shape = (self.k,initial_batch.shape[1]))
        self.labs = [self.kmeans.labels_]
        self.n_instances = [np.vstack(self.batches).shape[0]]
        self.error_conver = [self.errors[-1]/self.n_instances[-1]]
        self.error_init = []
        self.error_real_conver = [self.kmeans.inertia_]
        self.all_batches = [initial_batch]
        
        #If requested plot PCA or MDS projections of the first batch
        if "pca" in plot:
            labels = self.labs[-1]
            labels+= 1
            labels = labels/np.max(labels)
            cm = matplotlib.cm.get_cmap(cmap)
            self.pca = PCA(n_components = 2).fit(initial_batch)
            projection = self.pca.fit_transform(initial_batch)
            plt.scatter(x=projection[:,0],y=projection[:,1],c = cm(labels))
            plt.ylabel("PC2",size=20)
            plt.xlabel("PC1",size=20)
            plt.title("PCA projection: t = "+str(self.t[-1]+1),size=20)
            plt.savefig(os.path.join(os.getcwd(),"Projections","PCA_"+str(self.t[-1]+1)+".pdf"),format="pdf")
        
        if "mds" in plot:
            labels = self.labs[-1]
            labels+= 1
            labels = labels/np.max(labels)
            cm = matplotlib.cm.get_cmap(cmap)
            self.mds = MDS(n_components = 2).fit(initial_batch)
            projection = self.mds.fit_transform(initial_batch)
            plt.scatter(x=projection[:,0],y=projection[:,1],c = cm(labels))
            plt.ylabel("X2",size=20)
            plt.xlabel("X1",size=20)
            plt.title("MDS projection: t = "+str(self.t[-1]+1),size=20)
            plt.savefig(os.path.join(os.getcwd(),"Projections","MDS_"+str(self.t[-1]+1)+".pdf"),format="pdf")
            
    #This inner function is used each time a new batch arrives, and clusters the new batch and previous ones with different methods
        
    def Kmeans(self,batch,concept_drift = False,forget_centers = False,p=None,cd_index = 1,do_plot = False, plot = [""], cmap = "hsv", reset_projection = False,drift = False, hungaro = False,prev_centers = np.array([False])):
        
        
        ### Procedure removing or not previous information, prev_centers is given so that C^* is the same for every FSKM initialization method
        
        if prev_centers.any() and self.method != "minimize_real":
            self.centers[-1] = prev_centers
            self.labs[-1] = compute_labs(np.vstack(self.batches),self.centers[-1]) #Recompute labels for future calculations
        self.all_batches.append(batch)
        self.t.append(self.t[-1]+1)
        self.drift.append(concept_drift)
        self.dis_lloyd.append(0)
        self.dis_init.append(0)
        self.lloyd_iter.append(0)
        
        start = time.time() #t_0 to measure elapsed time
                
        ########### INITIALIZATION ################### 
        
        ## HI initialization ##
        
        if self.method == "weights": 
            
            w_prev = np.zeros((self.k))
            i = self.labs[-1].shape[0]
            t = 0

            while t<len(self.batches):
                w = self.rho[-2]**(t+1)
                batch_size = self.batches[len(self.batches)-t-1].shape[0]
                labs = self.labs[-1][i-batch_size:i]
                i-=batch_size
                u,c = count(labs,self.k)
                for lab in range(self.k):
                    w_prev[lab]+=w*c[lab]                    
                t+=1

            if hungaro:
                p = np.zeros((self.k,self.k))
                f = np.zeros((self.k,self.k))
                previous_cluster_p = w_prev

                km = KMeans(n_clusters = self.k,init = kplus(batch,self.k),n_jobs = self.jobs)
                km.fit(batch)
                self.dis_init[-1]+=dis_init_kmplus(n = batch.shape[0],k = self.k)
                self.dis_init[-1]+=km.n_iter_*self.k*batch.shape[0]
                #self.dis_init[-1]+=(len(self.batches)+1)*self.k**2
                u,new_cluster_p = count(km.labels_,k = self.k)
                
                for k in range(self.k):
                    for kp in range(self.k):
                        p[k,kp]=1/(1+new_cluster_p[kp]/(previous_cluster_p[k]+1.0e-10))
                        f[k,kp]+=(new_cluster_p[kp]*p[k,kp]*np.linalg.norm(km.cluster_centers_[kp]-self.centers[-1][k])**2)

                a = scipy.optimize.linear_sum_assignment(f)
                assignment = a[1]

                init_centers = np.zeros((self.k,batch.shape[1]))
                for k in range(self.k):
                    kp = assignment[k]
                    init_centers[k,:]=p[k,kp]*self.centers[-1][k]+(1-p[k,kp])*km.cluster_centers_[kp]

            else:
                p = np.zeros((self.k,self.k))
                previous_cluster_p = np.sum(temp_cluster_sizes,axis = 0)

                km = KMeans(n_clusters = self.k,init = self.centers[-1],n_jobs = self.jobs,max_iter = 1)
                km.fit(batch)
                self.dis_init[-1]+=km.n_iter_*self.k*batch.shape[0]
                u,new_cluster_p = count(km.labels_,k = self.k)
                for k in range(self.k):
                    kp = k
                    p[k,kp]=1/(1+new_cluster_p[kp]/(previous_cluster_p[k]+1.0e-10))

                init_centers = np.zeros((self.k,batch.shape[1]))
                for k in range(self.k):
                    kp = k
                    init_centers[k,:]=p[k,kp]*self.centers[-1][k]+(1-p[k,kp])*km.cluster_centers_[kp]            
            
            self.kmeans = KMeans(n_clusters = self.k,init = init_centers,n_jobs = self.jobs)
        
        ## WKI initialization ##
        
        if self.method == "centroid_kmeans":
 
            w_prev = np.zeros((self.k))
            i = self.labs[-1].shape[0]
            t = 0
            while t<len(self.batches):
                w = self.rho[-2]**(t+1)
                batch_size = self.batches[len(self.batches)-t-1].shape[0]
                labs = self.labs[-1][i-batch_size:i]
                i-=batch_size
                u,c = count(labs,self.k)
                for lab in range(self.k):
                    w_prev[lab]+=w*c[lab]                    
                t+=1
            km = KMeans(n_clusters = self.k,init = kplus(batch,self.k),n_jobs = self.jobs)
            km.fit(batch)
            self.dis_init[-1]+=dis_init_kmplus(n = batch.shape[0],k = self.k)
            self.dis_init[-1]+=km.n_iter_*self.k*batch.shape[0]
            u,w_0 = count(km.labels_,k = self.k)
            center_km = KMeans(n_clusters = self.k,init = kplus(np.concatenate((self.centers[-1],km.cluster_centers_),axis = 0),self.k,w=np.concatenate((w_prev,w_0),axis = 0)),n_jobs = self.jobs)
            self.dis_init[-1]+=dis_init_kmplus(n = 2*self.k,k = self.k)
            center_km.fit(np.concatenate((self.centers[-1],km.cluster_centers_),axis = 0),sample_weight = np.concatenate((w_prev,w_0),axis = 0))
            init_centers = center_km.cluster_centers_
            
            self.dis_init[-1]+= (center_km.n_iter_*2*self.k**2)
            
            self.kmeans = KMeans(n_clusters = self.k,init = init_centers,n_jobs = self.jobs)
            
        ## Update stored batches, and compute indexes of the recent concept ##
        if len(self.batches) == self.maxBatch:
            del self.batches[0]
        self.batches.append(batch)
        self.labs[-1] = compute_labs(np.vstack(self.batches),self.centers[-1])
        real_index = [x for x in range(-cd_index,0,1)]
        
        ## PSKM algorithm ##
        
        if self.method == "minimize_real": 
            if cd_index != 1:
                init_centers = self.centers[-1] #Minimizes real SKM error but initializes with previous centroids if a CD did not happen
                
            else:
                km = KMeans(n_clusters = self.k,init = kplus(batch,self.k),n_jobs = self.jobs)
                km.fit(batch)
                self.dis_init[-1]+=dis_init_kmplus(n = batch.shape[0],k = self.k)
                self.dis_init[-1]+=km.n_iter_*self.k*batch.shape[0]
                init_centers = km.cluster_centers_
                self.kmeans = KMeans(n_clusters = self.k,init = init_centers,n_jobs = self.jobs)
                self.dis_init[-1]+= dis_init_kmplus(n = batch.shape[0],k = self.k)
            self.kmeans = KMeans(n_clusters = self.k,init = init_centers,n_jobs = self.jobs)
            
        ## UPC initialization ##
            
        if self.method == "prev_centers": 
            init_centers = self.centers[-1]
            self.kmeans = KMeans(n_clusters = self.k,init = init_centers,n_jobs = self.jobs)   
            
        ## ICB initialization ##
        
        if forget_centers: #If chosen to forget, then for each new batch novel initial centers are computed
            if self.mini_batch_size:
                self.kmeans = MiniBatchKMeans(n_clusters = self.k,n_init = self.n_init,init = self.first_init,batch_size = self.mini_batch_size)
                
            elif self.first_init == "kmc2":
                self.kmeans = KMeans(n_clusters = self.k,n_init = self.n_init,init = kmc2(np.vstack(self.batches),
                            k = self.k, chain_length=self.chain_length, afkmc2=self.afkmc2,
                            weights = np.array([self.rho[i] for i in range(-len(self.batches),0,+1) for _ in range(self.batches[i].shape[0])])),
                            n_jobs = self.jobs)
                self.dis_init[-1]+= dis_init_kmc2(m = self.chain_length,k = self.k)
                
            else:
                km = KMeans(n_clusters = self.k,init = kplus(batch,self.k),n_jobs = self.jobs)
                km.fit(batch)
                self.dis_init[-1]+=dis_init_kmplus(n = batch.shape[0],k = self.k)
                self.dis_init[-1]+=km.n_iter_*self.k*batch.shape[0]
                init_centers = km.cluster_centers_
                self.kmeans = KMeans(n_clusters = self.k,init = init_centers,n_jobs = self.jobs)
                self.dis_init[-1]+= dis_init_kmplus(n = batch.shape[0],k = self.k)
        
        
        ### LLOYD
        ## Fit to the new data ##        
        
        if self.method == "minimize_real": #Lloyd over SKM error
            self.kmeans.fit(np.vstack(np.array(self.all_batches)[real_index,:]))
        
        else: #Lloyd over surrogate error
            self.kmeans.fit(np.vstack(self.batches),
                            sample_weight = np.array([self.rho[i] for i in range(-len(self.batches),0,+1) for _ in range(self.batches[i].shape[0])]))
        
        self.time.append(time.time()-start) #Compute elapsed time
        
        self.error_real_init.append(inertia(x = np.vstack(np.array(self.all_batches)[real_index,:]), centers = init_centers)) #Initial SKM error
        self.error_init.append(inertia(x = np.vstack(self.batches),centers = init_centers,weight = np.array([self.rho[i] for i in range(-len(self.batches),0,+1) for _ in range(self.batches[i].shape[0])]))) #Initial surrogate error
        
        ## Save results ##
        
        self.labs.append(self.kmeans.labels_)
        self.centers.append(self.kmeans.cluster_centers_)
        self.errors.append(self.kmeans.inertia_)
        self.error_ratio.append(self.errors[-1]/self.errors[-2])
        self.dis_lloyd[-1]+=(self.kmeans.n_iter_*self.k*np.vstack(self.batches).shape[0])
        self.dis.append(self.dis_lloyd[-1]+self.dis_init[-1])
        self.lloyd_iter[-1]+=(self.kmeans.n_iter_)
        self.n_instances.append(np.vstack(self.batches).shape[0])
        self.error_conver.append(inertia(x = np.vstack(self.batches),centers = self.centers[-1],weight = np.array([self.rho[i] for i in range(-len(self.batches),0,+1) for _ in range(self.batches[i].shape[0])])))
        self.error_real_conver.append(inertia(x = np.vstack(np.array(self.all_batches)[real_index,:]), centers = self.centers[-1]))
        
        if "pca" in plot and do_plot: #If stated plot projection
            labels = self.labs[-1]
            labels+= 1
            labels = labels/np.max(labels)
            cm = matplotlib.cm.get_cmap(cmap)
            if reset_projection:
                self.pca = PCA(n_components = 2).fit(np.vstack(self.batches))
            projection = self.pca.fit_transform(np.vstack(self.batches))
            plt.scatter(x=projection[:,0],y=projection[:,1],c = cm(labels))
            plt.ylabel("PC2",size=20)
            plt.xlabel("PC1",size=20)
            if drift:
                plt.title("PCA projection(Drift occurred): t = "+str(self.t[-1]+1),size=20)
            else:
                plt.title("PCA projection: t = "+str(self.t[-1]+1),size=20)
            plt.savefig(os.path.join(os.getcwd(),"Projections","PCA_"+str(self.t[-1]+1)+".pdf"),format="pdf")
        
        if "mds" in plot and do_plot:
            labels = self.labs[-1]
            labels+= 1
            labels = labels/np.max(labels)
            cm = matplotlib.cm.get_cmap(cmap)
            if reset_projection:
                self.mds = MDS(n_components = 2).fit(np.vstack(self.batches))
            projection = self.mds.fit_transform(np.vstack(self.batches))
            plt.scatter(x=projection[:,0],y=projection[:,1],c = cm(labels))
            plt.ylabel("X2",size=20)
            plt.xlabel("X1",size=20)
            if drift:
                plt.title("MDS projection(Drift occurred): t = "+str(self.t[-1]+1),size=20)
            else:
                plt.title("MDS projection: t = "+str(self.t[-1]+1),size=20)
            plt.savefig(os.path.join(os.getcwd(),"Projections","MDS_"+str(self.t[-1]+1)+".pdf"),format="pdf")
        
    def save(self,folder,name):
        
        ## Delete most heavy data in the object and save it as pyhton object ##
        
        del self.kmeans
        del self.batches
        del self.all_batches
        with open(os.path.join(folder,name), 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)
        
    def __str__(self):
        
        ## Return a string containing important information about the experiment procedure(parameters) ##
        
        n=40
        if self.jobs:
            if self.jobs < 0:
                return "\n".join(["Streaming Kmeans:\n","-"*n,"Data dimension = %s" % self.batches[0].shape[1],
                          "Number of clusters(k) = %s" % self.k,"Number of jobs(paralelization) = All","First initialization: %s" % self.first_init,"-"*n,
                          "Forget parameter: "+"rho"+"= "+str(self.rho[-2]),"Threshold: "+"epsilon"+"= "+str(self.thresh),
                          "Maximum number of batches stored = %s" % self.maxBatch,"Number of processed batches = %s" %(self.t[-1]+1)])
            else:
                return "\n".join(["Streaming Kmeans:\n","-"*n,"Data dimension = %s" % self.batches[0].shape[1],
                          "Number of clusters(k) = %s" % self.k,"Number of jobs(paralelization) = %s" % self.jobs,"First initialization: %s" % self.first_init,"-"*n,
                          "Forget parameter: "+"rho"+"= "+str(self.rho[-2]),"Threshold: "+"epsilon"+"= "+str(self.thresh),
                          "Maximum number of batches stored = %s" % self.maxBatch,"Number of processed batches = %s" %(self.t[-1]+1)])
        else:
            return "\n".join(["Streaming Kmeans:\n","-"*n,"Data dimension = %s" % self.batches[0].shape[1],
                          "Number of clusters(k) = %s" % self.k,"Number of jobs(paralelization) = 1","First initialization: %s" % self.first_init,"-"*n,
                          "Forget parameter: "+"rho"+"= "+str(self.rho[-2]),"Threshold: "+"epsilon"+"= "+str(self.thresh),
                          "Maximum number of batches stored = %s" % self.maxBatch,"Number of processed batches = %s" %(self.t[-1]+1)])
    
    def summary(self):
        ## Print a summary ##
        print(self)
    
        

### Boxplot

In [ ]:
## Define functions in order to plot results ##

def adapt_data(data=np.array([]),cd_per = 8): #Takes a data(stored over time), and modulates it to the concept drift period and can be plotted in boxplot format
    
    new_data = np.zeros(shape = (data.shape[1]//cd_per,cd_per,data.shape[0]))
    maximum = (data.shape[1]//cd_per)*cd_per
    for T in range(data.shape[0]):
        for t in range(cd_per):
            i = t
            index = list()
            while i<maximum:
                index.append(i)
                i+= cd_per
            new_data[:,(t)%cd_per,T] = data[T,index]
    return new_data   

#The data to be plotted is stored in dictionaries. Each experiment has a dictionary, being the keys the names of hyperparameters and values its values. Keys are also related to the measurements, 
#and the values are lists of the measures(initial error,elapsed time,...). xaxis is a dictionary with a single key, being the string to be plotted in the x axis, and the values to be stated as 
#labels for this axis. Folder where the output pdf files should be saved. plot is a list of strings, which contains what measurements we want to plot. cm is the colormap. identity helps to 
#differentiate the output apart from hyperparameter values on the title, save parameter works similarly but for the name of the saved file. scale accepts a dictionary with measures as keys
#and desired scales as values(default linear). Alpha determines the relative position of label ticks, should not be modified. ratio_ref is used if ratio is in plot list. cd_per 
#is the period in which a concept is stable. rho_list is the list of forget parameters used, digit_r determines to which digit is round up when used as string. showfliers determines whether to
#show outliers or not. figsize determines the figure size, and epsilon is the value of epsilon during the experiments.

def kmBoxplot(data=[],xaxis={},folder=None,plot=[None],cm="hsv",identity="",save = "",scale=dict(),alpha=1,
              ratio_ref = "Minimize real error",cd_per = 8,rho_list = [8,16], digit_r = 3, showfliers = True, figsize = (9,6),epsilon = 0.1):
    
    n_data = len(data)
    if n_data%2==0:
        relative=[i for i in range(-int(n_data/2),0)]
        relative.extend([i for i in range(1,int(n_data/2)+1)])
    else:
        relative=[i-n_data//2 for i in range(n_data)]
        
    ## Titles and y axis labels for each measurement ##
    
    title_dic = {"surrogate_real":"Real and surrogate error difference histogram","time":"Elapsed time","error_real_conver":"Real error in convergence","error_real_init":"Real initialization error","error_init":"Initialization error comparison","ratio":"Error ratio","error_conver":"Converged error comparison","n_iter":"Number of iterations comparison","n_dis":"Computed distances comparison","n_dis_lloyd":"Computed distances during Lloyd","n_dis_init": "Computed distances during initialization"}
    ylab_dic = {"surrogate_real":"Density","time":"Elapsed time(s)","error_real_conver":"Normalized error","error_real_init":"Normalized error","error_init": "Normalized error","ratio":"Error ratio","error_conver":"Normalized error","n_iter":"N iter","n_dis":"N distances","n_dis_lloyd":"N distances","n_dis_init":"N distances"}
    
    ## Run through all requested plots ##
    
    for pl in plot:
        
        if pl in scale.keys():
            sc = scale[pl]
        else:
            sc = "linear"
        if pl == "surrogate_real" or pl == "ratio":
            continue
        
        for T in range(len(rho_list)):
            
            r = rho_list[T]
                
            fig = plt.figure(1, figsize=figsize)
            ax = fig.add_subplot(111) #Create an axes instance
            cmap = matplotlib.cm.get_cmap(cm)  #Create a colormap instance

            b = [None for _ in range(len(data))] #Boxplot plots list

            for i in range(len(data)):
                col = cmap(i/(len(data))) #Define color for each method
                pos1 = np.array([value for value in xaxis.values()][0]) #Compute positions for each boxplot
                pos = alpha*len(data)*pos1
                for j in range(len(pos)):
                    pos[j]=pos[j]+relative[i]
                box = adapt_data(data[i][pl],cd_per)[:,:,T]

                ## Plot each boxplot ##
                b[i] = ax.boxplot(box,patch_artist=True,
                            positions=pos,boxprops=dict(facecolor=col,color="black"),
                            whiskerprops=dict(color=col,linestyle="--"),capprops=dict(color="black"),medianprops=dict(color="black"),
                            flierprops=dict(marker="o",color=col,alpha=1),manage_ticks = False, showfliers = showfliers)
                for flier in b[i]["fliers"]:
                    flier.set_markerfacecolor(col)
                    
            ## Set axis parameters and text ##
            ax.set_xticks(alpha*len(data)*pos1)
            ax.set_xticklabels([str(x) for x in [value for value in xaxis.values()][0]])
            ax.get_yaxis().tick_left()
            ax.legend([box["boxes"][0] for box in b], [ dat["name"] for dat in data], loc='upper right')
            ax.set_yscale(sc)
            pos = [value for value in xaxis.values()][0]
            if identity!="":
                ax.set_title(title_dic[pl]+": "+identity,fontsize="xx-large")
            else:
                ax.set_title(title_dic[pl],fontsize="xx-large")
            ax.set_ylabel(ylab_dic[pl])
            ax.set_xlabel([value for value in xaxis.keys()][0])
            
            ## Draw vertical lines to separate each index ##
            for i in range(len(pos1)-1):
                ax.axvline(x = alpha*len(data)*(pos1[i]+pos1[i+1])/2,linestyle = "-.",color = "grey",linewidth = 1)

            ## Save plots as pdf ##
            if folder:
                if save:
                    fig.savefig(os.path.join(folder,pl,pl+save+".pdf"),format="pdf")
                else:
                    fig.savefig(os.path.join(folder,pl,pl+identity+".pdf"),format="pdf")
            else:
                fig.savefig(os.path.join(parent(path),"Results",pl,pl+identity+", "+"rho"+" = "+str(round(r,3))+", "+"epsilon"+" = "+str(epsilon)+".pdf"),format="pdf")
            fig.clf()    
    
    if "surrogate_real" in plot: #This one is computes relative differences between surrogate and SKM error
        pl = "surrogate_real"
        for T in range(len(rho_list)):
            r = rho_list[T]
            data_list = np.array([])
            for i in range(len(data)):
                Data = abs(data[i]["error_real_conver"]-data[i]["error_conver"])/data[i]["error_real_conver"]
                Data = Data[T,:]
                data_list = np.concatenate((data_list,Data),axis = 0)
            if len(data_list)==0:
                continue
            fig = plt.figure(1, figsize=figsize)
            ax = fig.add_subplot(111)
            ax.hist(np.log(data_list)/np.log(10),color = "teal",bins = 100)
            ax.set_title(title_dic[pl]+identity,fontsize="xx-large")
            ax.set_ylabel(ylab_dic[pl])
            ax.set_xlabel(r"$\frac{|E_T-E_{\rho}|}{E_T}$",fontsize="x-large")
            if folder:
                fig.savefig(os.path.join(folder,pl,pl+identity+".pdf"),format="pdf")
            else:
                fig.savefig(os.path.join(parent(path),"Results",pl,pl+identity+", "+"rho"+" = "+str(round(r,3))+".pdf"),format="pdf")
            fig.clf()
        
    if "ratio" in plot: #Plots measurements in ratios compared to the one determined by ratio_ref
    
        pl = "ratio"
        k = np.where(np.array([ dat["name"] for dat in data]) == ratio_ref)[0][0]
        ref = data.pop(k)
        
        for error in ["error_conver","error_init","error_real_conver","error_real_init"]:
            for T in range(len(rho_list)):
                
                r = rho_list[T]
                fig = plt.figure(1, figsize=figsize)

                ## Create an axes instance ##
                ax = fig.add_subplot(111)
                cmap = matplotlib.cm.get_cmap(cm)

                b = [None for _ in range(len(data))]
                
                for i in range(len(data)):
                    col = cmap(i/(len(data)))
                    pos1 = np.array([value for value in xaxis.values()][0])
                    pos = alpha*len(data)*pos1
                    for j in range(len(pos)):
                        pos[j]=pos[j]+relative[i]
                    box = adapt_data((data[i][error]-ref[error])/ref[error],cd_per)[:,:,T]
                    b[i] = ax.boxplot(box,patch_artist=True,
                                positions=pos,boxprops=dict(facecolor=col,color="black"),
                                whiskerprops=dict(color=col,linestyle="--"),capprops=dict(color="black"),medianprops=dict(color="black"),
                                flierprops=dict(marker="o",color=col,alpha=1),manage_ticks = False, showfliers = showfliers)
                    for flier in b[i]["fliers"]:
                        flier.set_markerfacecolor(col)
                ax.set_xticks(alpha*len(data)*pos1)
                ax.set_xticklabels([str(x) for x in [value for value in xaxis.values()][0]])
                ax.get_yaxis().tick_left()
                ax.legend([box["boxes"][0] for box in b], [ dat["name"] for dat in data if dat["name"]!=ratio_ref], loc='lower right')
                ax.set_yscale(scale)
                for xv in alpha*len(data)*pos1:

                    ax.axvline(x = xv,linestyle = "-.",color = "grey",linewidth = 1)

                if identity!="":
                    ax.set_title(title_dic[pl]+"("+error+"): "+identity,fontsize="xx-large")
                else:
                    ax.set_title(title_dic[pl]+"("+error+")",fontsize="xx-large")
                ax.set_ylabel(ylab_dic[pl])
                ax.set_xlabel([value for value in xaxis.keys()][0])
                if folder:
                    fig.savefig(os.path.join(folder,pl,pl+error+identity+".pdf"),format="pdf")
                else:
                    fig.savefig(os.path.join(parent(path),"Results",pl,pl+error+identity+", "+"rho"+" = "+str(round(r,3))+".pdf"),format="pdf")
                fig.clf()      
    

In [ ]:
## This function plots the results of the experiment where the surrogate is compared to the SKM error ##

#Data contains the measured differences between both error functions, up and low contain theoretical upper and lower bounds. 
#Every other parameter work similar to the previous function, plus there is another parameter N which states the batch size of the experiment.

def SurrogateBoxplot(data=[],up=[],low=[],xaxis={},folder=None,r = 0.1,epsilon=0.1,cm="hsv",identity="",scale="linear",alpha=1,
                     showfliers = True, figsize = (16,9), N = 10,delta_list = []):
                
    identity = "Simulated Data"
    fig = plt.figure(1, figsize=figsize)

    ## Create an axes instance ##
    ax = fig.add_subplot(111)
    cmap = matplotlib.cm.get_cmap(cm)
    col = cmap(0.4)
    pos1 = np.array([value for value in xaxis.values()][0])
    s = 10
    ## Plot boxplots ##
    b = ax.boxplot(data,patch_artist=True,
                positions=pos1,boxprops=dict(facecolor=col,color="black"),
                whiskerprops=dict(color=col,linestyle="--"),capprops=dict(color="black"),medianprops=dict(color="black"),
                flierprops=dict(marker="o",color=col,alpha=1),manage_ticks = False, showfliers = showfliers)
    
    for flier in b["fliers"]:
        flier.set_markerfacecolor(col)
    ax.tick_params(axis='both', which='major', labelsize=s)
    x = np.linspace(pos1[0],pos1[-1],100)
    gauss = [0 for i in range(up.shape[0])]
    gauss_label = [Patch(facecolor = "b",alpha = (i+1)/(up.shape[0]+1),label = "%s"%(int(100*(1-delta_list[i])))+"\% confidence interval") for i in range(up.shape[0])]
    ## Interpolate lines for upper and lower bounds and fill the space between them ##
    for i in range(up.shape[0]):
        upper = up[i,:]
        alpha = (i+1)/(up.shape[0]+1)
        interpolate = scipy.interpolate.make_interp_spline(pos1, upper)
        interpolate_upper = interpolate(x)
        l1 = ax.plot(x,interpolate_upper,alpha=alpha, color='b')
        lower = low[i,:]
        interpolate = scipy.interpolate.make_interp_spline(pos1, lower)
        interpolate_lower = interpolate(x)
        l2 = ax.plot(x,interpolate_lower,alpha=alpha, color='b')
        gauss[i] = plt.fill(np.concatenate([x, x[::-1]]),
             np.concatenate([interpolate_upper,
                            (interpolate_lower)[::-1]]),
             alpha=alpha, fc='b', ec='None')
    
    ## Axis setup ##
    ax.set_xticks(pos1)
    ax.set_xticklabels([str(x) for x in [value for value in xaxis.values()][0]],fontsize=s)
    ax.get_yaxis().tick_left()
    legend_elements = [Patch(facecolor = col,label = "Observations")]
    legend_elements.extend(gauss_label)
    s = 60
    ## Use tex fonts ##
    plt.rcParams.update({
    "text.usetex": True,
    "font.weight":"light"
})
    font = {
    "fontweight":"light",
    "fontsize":s
}
    ax.legend(handles=legend_elements,loc='upper right',fontsize=35)
    ax.tick_params(axis='both', which='major', labelsize=25)
    ax.set_yscale(scale)
    ax.set_xlabel(r"Number of batches since last drift, $T$",fontdict=font)
    ax.set_ylabel(r"$E_*-E_{\rho}$",fontdict=font)
    ax.set_ylim(-1500,7500) #Set limits of y axis, same limits for each every experiment for comparability
    if identity!="":
        ax.set_title(r"$\rho$"+" = "+str(round(r,3))+", "+r"$N$"+" = "+str(N),fontdict=font)
    else:
        ax.set_title("rho"+" = "+str(round(r,3))+", "+"epsilon"+" = "+str(epsilon)+", N = "+str(N),fontsize=s)
    
    ## Save figure ##
    if folder:
        fig.savefig(os.path.join(folder,identity+", "+"rho"+" = "+str(round(r,3))+", "+"epsilon"+" = "+str(epsilon)+", N = "+str(N)+".pdf"),format="pdf")
    else:
        fig.savefig(os.path.join(parent(path),"Surrogate",identity+", "+"rho"+" = "+str(round(r,3))+", "+"epsilon"+" = "+str(epsilon)+", N = "+str(N)+".pdf"),format="pdf")
    fig.clf() 